# Step 6: 데이터셋 검증

전처리된 데이터셋의 품질을 검증합니다.

## 왜 검증이 필요한가?
전처리 후 데이터셋에 문제가 없는지 확인해야 합니다:
- 이미지와 라벨이 1:1로 매칭되는가?
- 라벨 파일이 누락된 이미지는 없는가?
- 클래스 분포가 적절한가? (불균형이 심하면 학습에 영향)
- 바운딩 박스가 올바르게 그려지는가?

## 검증 항목
1. **이미지-라벨 매칭 확인**
   - 이미지는 있는데 라벨이 없는 경우 (orphan image)
   - 라벨은 있는데 이미지가 없는 경우 (orphan label)

2. **클래스 분포 분석**
   - 각 클래스별 객체 수 계산
   - Train/Val/Test 분포 확인

3. **시각화**
   - 랜덤 샘플에 바운딩 박스를 그려서 확인
   - 라벨이 올바르게 적용되었는지 육안 검증

## 입력/출력
- 입력: images/train/, images/val/, images/test/
- 출력: images/processed/samples/ (시각화 이미지)

In [ ]:
from pathlib import Path
from collections import Counter
import cv2
import random
from IPython.display import Image, display

## 클래스 정의

In [ ]:
# 클래스 ID와 이름 매핑
CLASS_NAMES = {0: 'helmet', 1: 'vest'}

# 시각화 시 사용할 색상 (BGR 형식)
CLASS_COLORS = {0: (0, 255, 0), 1: (255, 165, 0)}  # 초록, 주황

## 경로 설정

In [ ]:
# 프로젝트 루트 디렉토리
base_dir = Path.cwd().parent
images_dir = base_dir / 'images'

print(f"이미지 경로: {images_dir}")

## 데이터셋 검증

In [ ]:
splits = ['train', 'val', 'test']
total_stats = {'images': 0, 'labels': 0, 'objects': Counter()}

print("=" * 50)
print("Step 6: 데이터셋 검증")
print("=" * 50)
print()

for split in splits:
    split_dir = images_dir / split
    img_dir = split_dir / 'images'
    lbl_dir = split_dir / 'labels'
    
    if not split_dir.exists():
        print(f"⚠️  {split} 폴더가 없습니다.")
        continue
    
    # 파일 목록
    images = set(
        p.stem for p in img_dir.glob('*')
        if p.suffix.lower() in ['.jpg', '.jpeg', '.png']
    )
    labels = set(p.stem for p in lbl_dir.glob('*.txt'))
    
    # 매칭 확인
    matched = images & labels
    img_only = images - labels
    lbl_only = labels - images
    
    print(f"📁 {split.upper()}")
    print(f"   이미지: {len(images)}개")
    print(f"   라벨: {len(labels)}개")
    print(f"   매칭됨: {len(matched)}개")
    
    if img_only:
        print(f"   ⚠️  라벨 없는 이미지: {len(img_only)}개")
    if lbl_only:
        print(f"   ⚠️  이미지 없는 라벨: {len(lbl_only)}개")
    
    # 클래스 분포
    class_count = Counter()
    for lbl_file in lbl_dir.glob('*.txt'):
        with open(lbl_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if parts:
                    class_id = int(parts[0])
                    class_count[class_id] += 1
                    total_stats['objects'][class_id] += 1
    
    print(f"   클래스 분포:")
    for class_id, count in sorted(class_count.items()):
        class_name = CLASS_NAMES.get(class_id, f'unknown_{class_id}')
        print(f"      - {class_name}: {count}개")
    
    total_stats['images'] += len(matched)
    total_stats['labels'] += len(labels)
    print()

## 전체 통계

In [ ]:
print("=" * 50)
print("📊 전체 통계")
print("=" * 50)
print(f"총 이미지: {total_stats['images']}개")
print(f"총 객체:")
for class_id, count in sorted(total_stats['objects'].items()):
    class_name = CLASS_NAMES.get(class_id, f'unknown_{class_id}')
    print(f"   - {class_name}: {count}개")

## 샘플 이미지 시각화

In [ ]:
# 출력 디렉토리
output_dir = base_dir / 'images' / 'processed' / 'samples'
output_dir.mkdir(parents=True, exist_ok=True)

# 훈련 이미지 목록
train_images = list((images_dir / 'train' / 'images').glob('*'))

if train_images:
    # 랜덤 샘플 선택
    num_samples = 5
    samples = random.sample(train_images, min(num_samples, len(train_images)))
    
    print("\n🖼️  샘플 이미지 시각화")
    print("=" * 50)
    
    for i, img_path in enumerate(samples):
        img = cv2.imread(str(img_path))
        if img is None:
            continue
        
        h, w = img.shape[:2]
        
        # 라벨 파일
        lbl_path = images_dir / 'train' / 'labels' / (img_path.stem + '.txt')
        if lbl_path.exists():
            with open(lbl_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        x_center = float(parts[1]) * w
                        y_center = float(parts[2]) * h
                        box_w = float(parts[3]) * w
                        box_h = float(parts[4]) * h
                        
                        x1 = int(x_center - box_w / 2)
                        y1 = int(y_center - box_h / 2)
                        x2 = int(x_center + box_w / 2)
                        y2 = int(y_center + box_h / 2)
                        
                        color = CLASS_COLORS.get(class_id, (255, 255, 255))
                        class_name = CLASS_NAMES.get(class_id, f'class_{class_id}')
                        
                        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(img, class_name, (x1, y1 - 5),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # 저장
        output_path = output_dir / f'sample_{i+1}.jpg'
        cv2.imwrite(str(output_path), img)
        print(f"  저장: {output_path.name}")
    
    print(f"\n✅ {len(samples)}개 샘플 이미지 저장 완료!")
    print(f"   - 출력 위치: {output_dir}")
else:
    print("⚠️  훈련 이미지가 없습니다.")

## 저장된 샘플 이미지 확인

In [ ]:
# 저장된 샘플 이미지 표시
sample_files = sorted(output_dir.glob('sample_*.jpg'))
for sample_file in sample_files[:3]:  # 처음 3개만 표시
    print(f"\n{sample_file.name}:")
    display(Image(filename=str(sample_file), width=400))